# Criar imagens apartir de um container

## Docker Commit

__Recomendado apenas para ambientes de testes__

* Processo simplificado
* Consolida mudanças - container numa imagem
    * E por isso torna-se __resistente__ a mudanças já realizadas

* O Processo se da da seguinte forma:
    * Iniciar um container temporário
    * Realizar as mudanças
    * Consolidar a mudança ( _docker commit_ )
    * Novos containers apartir da imagem

__Sintaxe do comando:__

* __docker commit OPTIONS CONTAINER REPO__
    * __OPTIONS:__ --change, -c -> Aplica instruções DockerFile
    * adotar formato: __nomerepo/nomeimagem:versão__ --> _Boas práticas_

### Exemplo

* Criar um container temporario:
    * __docker run -it ubuntu__ (isso inicializará o bash o ubuntu pelo container)
    
    
* Dentro do container fazer alterações nele
    * Como por exemplo: __apt-get update && apt-get install figlet__


* Consolidar as mudanças (Commit)
    * __docker commit < ImageID > minhaimagem__ (Usando um apelido no final)
    
    
* Criar um container apartir da imagem criada
    * __docker run -it minhaimagem__
    
    
__OBS:__ É possível nomear/renomear a imagem após a criação dela com __docker tag < ImageID > NOVO_NOME__

## Docker Build

__Altamente recomendável para ambientes de produção__

* Automatiza o processo de criação de imagens
* Processo maduro (ambiente de produção)
* Controle de Desenvolvimento - _instructions_
* Gestão de Mudanças
* DockerFile - Mudanças num arquivo de texto
    * Automatiza processo de crianção de imagens 

__Etapas:__

* Criar o arquivo DockerFile
* Processo de Build
* Containers podem ser criados usando a imagem

__Processo de Build:__

* Validação de sintaxe
* _Build context_ - Considera o diretório em que o arquivo Dockerfile esta como um diretório de contexto
    * Isso é importante pois toda vez que aumentamos o espaço do diretório em que o arquivo Dockerfile esta, é feito uma busca por todo o diretório, recursivamente, por arquivos que poderão fazer parte da futura imagem, aumentando o contexto de execução dele, causando maior eficiencia.

* 1 Instrução -> 1 camada intermediaria no processo (Recomendado tentar reduzir a qtd de instruções)
* __cache Commits__ dos comandos RUN
    * É possível ignorar isso com __--no-cache=true__
* Cache de outras imagens geradas localmente
    * Incluir a opção __--cache-from repo/imagem:tag__

### Exemplo

Primeiro é importante criar um diretório para armazenar a dockerfile, é útil e faz parte das boas práticas

* Criar o Dockerfile
    * Em qualquer editor de texto pode ser feito, existem dezenas de Options que podem ser usadas e sempre após as options temos um shell script, como exemplo:
        * FROM debian
        * RUN apt-get update
        * RUN apt-get install apache2 -y
            * Com o __-y__ estamos forçando a instalação para que não seja necessário o usuario aperta "yes" durante a instalação
        
        
* Processo de build
    * __docker build -t NOMEDAIMAGEM/VERSÃO:0.0__
        * __-t__ é a options de _tag_ para nomear a imagem que sera criada
        
        
* Criação de um container usando a imagem criada
    * __docker run -id --name web2 -p 8082:80 dockerdozero/web2:2.0 /usr/sbin/apache2ctl -D FOREGROUND__
        * como não temos nenhum comando padrão passado no dockerfile teremos que inserir um _command arq_ : 
            * __/usr/sbin/apache2ctl -D FOREGROUND__

__OBS:__ Caso seja necessario fazer alguma mudança na imagem, como por exemplo trocar a imagem que será usada (ubuntu -> debian). Neste caso apenas editamos o arquivo Dockerfile e refazemos o build

## Dockerfile REFERENCE

![](dockerfile.png)

__Instruções:__

* __FROM__
    * Imagem:Versão
* __RUN__
    * executa qualquer comando na imagem passada (FROM) e confirma (COMMIT) resultados
    * Sintaxe:
        * Run < comando > (shell form - mais comum) (/bin/sh -c [LINUX] e cmd /S/C [WINDOWS])
            * __RUN apt-get install ...__
        * Ou
        * RUN [executable, param1, param2] (exec form)
            * __RUN ["apt-get","install", "apache2", "-y" ]__
            
     * Cria uma layer PERSISTENTE -> aumenta o tamanho final da imagem

* __copy__
    * Copia arquivos do diretório atual do host (build context) para dentro da imagem
        * __src__ - destino
        * cria uma layer persistente

* __add__
    * Similar a copy
    
* __workdir__
    * Configura o ambiente de trabalho, base para as instruções RUN, CMD, etc.., definindo o diretório de trabalho
    
* __expose__
    * expõe o socket de execução, apenas sinaliza a porta, não faz o mesmo que a option __-p__
    
* __volume__
    * informa qual será o volume criado
    * Dispensa a criação do _docker volume_ e o posterior mapeamento dele (__-v__)
    * Automaticamente isso é feito no Dockerfile

## Comando Padrão

* __CMD:__
    * Comando/argumento padrão a ser executado pelo container - __Sempre como a última instrução a ser passada__
    * Sintaxe:
        * __CMD ["executable", "param1", "param2"]__ (exec form - recomendado e mais comum)
        * __CMD command param1 param2__ (shell form -wrapping "/bin/sh -c");
        * É possível _sobrescrever_ a instrução CMD

* __ENTRYPOINY:__
    * Permite configurar o container para ser executado como um executável
        * __/bin/sh -c__ --> entrypoint padrão
        * normalmente possui CMD como argumento
        * Pode ser iniciado por um script executável ( _.sh_ )
        * Utiliza-se o formate _exec form_ (json)
      
      
* É possível alterar o Entrypoint exclusivamente para a execução do container em questão, forçando essa mudança, utilizando o __--entrypoint NOVO_ENTRY__ e sendo assim passada a imagem, será possível passar novas _options_, exclusivas do NOVO_ENTRY, que irão ser executadas apenas para o container em questão

![](compare.png)

__CMD é sobrescrito ENTRYPOINT NÂO!!__

### Exemplo:

__Executar um script em C__

Em um mesmo diretório armazenar o arquivo.c e Dockerfile

In [ ]:
#hello.c
int main(){
    puts("Hello World");
    return 0;
}

In [ ]:
#Dockerfile
FROM ubuntu
RUN apt-get update
RUN apt-get install -y build-essential
COPY hello.c /
RUN make hello
CMD /hello

__Com o arquivo .C copiado para dentro da imagem (que será gerada) podemos criar um programa em C utilizando um container__

* Criando o programa em C:
    * __docker build -t helloworld .__ 
        * O " . " indica que o Dockerfile esta no diretório corrente
        
* Criando o container com o programa C:
    * __docker run hello__ ( _hello --> nome do programa C_ )
        * __Saida:__ Hello World

## Boas práticas para escrever um Dockerfile

" ___Uma das coisas mais desafiadoras na criação de imagens é manter o seu tamanho reduzido___ "

### Containers Efêmeros

* Efêmero = Passagerio e Transitório
    * Container pode ser parado e destruido, depois reconstruidoe substituido por um minimo  absoluto de configurações
    

* VI. Processos conforme metodologia twelve-factors app
    * " _Execute a aplicação como um ou mais processos que não armazenam estado (etateless)_ "
    * " _Quaisquer dados que precise persistir deve ser armazenado em um serviço de apoio stateful (que armazena o seu estado), tipicamenteuma base de dados_ " --> Uso do Docker object volume

### De olho no Buid Context

É o __Diretório atual__ do build 

* Quanto maior o conteúdo da pasta do Dockerfile
    * Maior contexto de build, maior tamanho da imagem, maior tempo de build, push e pull, maior tamanho do container
    
* Sempre criar um diretório especifico para o arquivo Dockerfile
    * Ou utilizar o __.dockerignore__
    

* Um diretório para o Dockerfile e outro para arquivos da imagem ( _COPY_ )
    * Uso da _option_ __-f__

### Evitar pacotes desnecessários

* Reduzir a complexidade, o tamanho dos arquivos e o tempo de criação
    * Você não precisa incluir um editor de texto em uma imagem de banco de dados!
    
* Diminuir o número de layers
    * RUN, COPY e ADD geram layers persistentes - outras instruções geram layers intermediárias
    * Por exemplo, para diversos argumentos para o comando RUN, utilizar a concatenação deles (&& \) para gerar o minimo de layers possível

### Exemplo:

In [ ]:
#Dockerfile Padrão

FROM ubuntu:18.04
    
RUN apt-get update \
&& apt-get upgrade -y \
&& apt-get instaçç -y node.js npm \
&& rm -rf /var/lib/apt/lists/* #Remove o cache

COPY . .

RUN npm install

CMD ["node", "hello.js"]

___A imagem gerada tem ~385mb___

In [ ]:
#Dockerfile otimizado + boas praticas

#Não há necessidade de utilizar o ubuntu como base image para após isso instalar o node.js npm
#Já existe imagens oficiais do node

FROM node:12-alpine #Imagem muito utilizada por ser bem "enxuta", reduz bastante o tamanho da imagem

#Opcional, delimita um escopo de execução do diretório, caso ele não exista será então criado
WORKDIR /app

#Ao invés de fazer o COPY em tudo, nós limitamos o copy aqui (os packages json vão para dentro do diretório app)
#Lembrando que a execução é top -> down então instruções com pouca variação devem ficar em cima por conta cache,
#pois nos processos de builds futuros a execução utilizará deste cache
COPY package.json .

RUN npm install

#Agora sim copiamos a aplicação para dentro do diretório
#Aumenta o nível de cache
COPY hello.js .

#Deixamos a porta claro a porta exposta, reservamos o socket para esta aplicação, mesmo se usarmos o -P
EXPOSE 8080

CMD ["node", "hello.js"]

___A imagem gerada agora tem ~83mb, uma grande redução no tamanho e tempo de execução do build___

# Envio das imagens criadas para o docker hub

__Login:__

__Docekr ID:__ _cursodockerdozero_ <br>
__email:__ leandro.vieira.silva@usp.br <br>
__senha:__ afamosasenha

__É possível enviar as imagens para o Dockerhub pelo terminal__

* __docker login__
    * inserir login e senha

__Agora temos credenciais para envio__

* __docker push REPOSITORIO__
    * O repositório inserido aqui deve ser o mesmo criado no site
    * Caso a imagem tenha nome diferente do repositório criado no site, podemos renomear a imagem antes de fazer o envio
        * __docker tag IMAGEM NOVONOME__